In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# 1. Definición de constantes (Datos Reales de la Imagen)
COSTO_KWH_EPM = 765.37  #
MARGEN_MERCADO = 1.3
INC_TRIMESTRAL_MATERIAL = 1.05 # 5% de aumento

# Especificaciones de productos
productos = {
    'caja_1': {'t': 360, 'mat': 24, 'e': 0.025},
    'caja_2': {'t': 240, 'mat': 26, 'e': 0.016},
    'caja_3': {'t': 300, 'mat': 48, 'e': 0.020}
}

# 2. Generación de Población Aleatoria (Ventas de 3 meses)
np.random.seed(42)
registros = []

for mes in [1, 2, 3]:
    for _ in range(20): # 20 ventas por mes
        tipo = np.random.choice(['caja_1', 'caja_2', 'caja_3'])
        p = productos[tipo]

        # Costo base calculado
        # (Costo material base $0.15 + ajuste por mes)
        c_mat = 0.15 * (1.02 ** mes)
        costo_total = (p['t'] * 0.02) + (p['mat'] * c_mat) + (p['e'] * COSTO_KWH_EPM)

        registros.append([p['t'], p['mat'], p['e'], costo_total])

df_prod = pd.DataFrame(registros, columns=['tiempo_seg', 'material_cm2', 'energia_kwh', 'costo_real'])

# 3. Entrenamiento del Modelo de Producción
X = df_prod[['tiempo_seg', 'material_cm2', 'energia_kwh']]
y = df_prod['costo_real']
modelo_laser = LinearRegression().fit(X, y)

print(f"Modelo entrenado con datos de Antioquia. Precisión (R2): {modelo_laser.score(X, y):.4f}")


In [ ]:
!pip install onnx skl2onnx
!pip install onnxruntime

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType


# --- CONFIGURACIÓN DE DATOS REALES ---
COSTO_KWH_EPM = 765.37  # Valor extraído de tu factura

# Definición de tus productos específicos
specs = {
    'Caja 1 (Pequeña)': [360, 24, 0.025], # tiempo, mat, energia
    'Caja 2 (Mediana)': [240, 26, 0.016],
    'Caja 3 (Grande)':  [300, 48, 0.020]
}

# Simulación de población para entrenamiento (3 meses)
data = []
for mes in [1, 2, 3]:
    for producto, valores in specs.items():
        # Añadimos un ligero ruido para simular variaciones reales de operación
        costo_base = (valores[0] * 0.025) + (valores[1] * 0.15 * (1.05**mes)) + (valores[2] * COSTO_KWH_EPM)
        data.append(valores + [costo_base])

df = pd.DataFrame(data, columns=['tiempo_seg', 'material_cm2', 'energia_kwh', 'costo_real'])

# --- ENTRENAMIENTO ---
X = df[['tiempo_seg', 'material_cm2', 'energia_kwh']]
y = df['costo_real']
modelo_laser = LinearRegression()
modelo_laser.fit(X, y)

# --- RESULTADOS DEL TRIMESTRE ---
print("--- RESULTADOS DE COSTOS POR PRODUCTO (TRIMESTRE 1) ---")
for nombre, valores in specs.items():
    prediccion = modelo_laser.predict([valores])[0]
    precio_detal = prediccion * 1.3 # Tu margen de valoración de mercado
    print(f"{nombre}:")
    print(f"  > Costo Producción: ${prediccion:.2f} COP")
    print(f"  > Precio al Detal:  ${precio_detal:.2f} COP\n")

print(f"Precisión del Modelo (R2 Score): {modelo_laser.score(X, y):.4f}")
print(f"Intercepto (Costos Fijos): {modelo_laser.intercept_:.2f}")

# --- EXPORTACIÓN A ONNX ---
initial_type = [('float_input', FloatTensorType([None, 3]))]
onx = convert_sklearn(modelo_laser, initial_types=initial_type)

with open("costos_cajas_laser.onnx", "wb") as f:
    f.write(onx.SerializeToString())

print("\n[ÉXITO] Modelo 'costos_cajas_laser.onnx' generado para el microservicio.")

In [ ]:
import onnxruntime as ort
import numpy as np

# 1. Cargar el "cerebro" (Modelo ONNX)
# Este archivo debe estar en la misma carpeta
try:
    session = ort.InferenceSession("costos_cajas_laser.onnx")
    input_name = session.get_inputs()[0].name
    print("✅ Modelo ONNX cargado correctamente.")
except Exception as e:
    print(f"❌ Error al cargar el modelo: {e}")

# 2. Función para realizar la inferencia
def obtener_cotizacion(tiempo, area, energia):
    """
    Recibe los datos físicos y devuelve el costo predicho.
    Basado en el costo de EPM: $765.37 por kWh.
    """
    # Preparar los datos como un array de 32 bits (lo que pide ONNX)
    datos_entrada = np.array([[tiempo, area, energia]], dtype=np.float32)

    # Ejecutar la matemática del modelo
    resultado = session.run(None, {input_name: datos_entrada})
    costo_produccion = float(resultado[0][0])

    # Aplicar tu margen de valoración al mercado (30%)
    precio_detal = costo_produccion * 1.3

    return costo_produccion, precio_detal

# 3. Prueba con tus 3 modelos de cajas
print("\n--- SIMULADOR DE PRECIOS ANTIOQUIA 2026 ---")

# Datos: [tiempo_seg, material_cm2, energia_kwh]
cajas = {
    "Caja 1": [360, 24, 0.025],
    "Caja 2": [240, 26, 0.016],
    "Caja 3": [300, 48, 0.020]
}

for nombre, specs in cajas.items():
    costo, detal = obtener_cotizacion(specs[0], specs[1], specs[2])
    print(f"📦 {nombre}:")
    print(f"   Costo de energía aplicado: ${765.37 * specs[2]:.2f} COP")
    print(f"   Producción: ${costo:.2f} COP")
    print(f"   Venta sugerida: ${detal:.2f} COP\n")